# Zaawansowane Przetwarzanie Języka Naturalnego
## Laboratorium 3: Neuronowe modele predykcji sekwencji

Celem zadania jest zaimplementowanie neuronowego modelu do predykcji sekwencji i zastosowanie go do zadania płytkiej analizy frazowej. Zaimplementowany model będzie oparty o dwukierunkowe sieci rekurencyjne oraz zakończony będzie warstwą CRF. Celem ćwiczenia nie jest stworzenie wydajnej implementacji odpowiedniej do pracy z dużymi zbiorami treningowymi, ale pogłębienie zrozumienia działania metod modelowania predykcji poprzez ich implementację.

## Zadanie 1
Pracując w środowisku PyTorch nie jest konieczne samodzielne implementowanie neuronów rekurencyjnych, gdyż biblioteka `torch.nn` zawiera ich gotowe implementacje. Klasyczny neuron rekurencyjny z funkcją aktywacji tangens hiperboliczny jest zaimplementowany w klasie `RNN` i od razu pozwala na przetworzenie od razu całej sekwencji elementów. Konstruktor obiektu wymaga wyspecyfikowania liczby cech którymi opisany jest każdy element sekwencji, liczby neuronów w warstwie rekurenycjnej oraz liczbę warstw rekurencyjnych. 

Istotnym parametrem modelu jest opcjonalny przełącznik `batch_first = True` który określa format wejściowej sekwencji. Domyślnie modele rekurencyjne spodziewają się wejścia o wymiarach `(długość sekwencji, rozmiar batcha, liczba cech)` czyli w kolumnach tensora mamy kolejne sekwencje (przykłady uczące), w wierszach kolejne elementy tych sekwencji, a w głębokości tensora umieszone są kolejne cechy każdego z elementów sekwencji. Zwróć uwagę, że jest to format inny od często stosowanego formatu danych niesekwencyjnych gdzie w wierszach umieszcza się kolejne przykłady uczące, a w kolumnach kolejne cechy (tutaj jest na odwrót: przykład uczący-sekwencja umieszcona jest w kolumnie, a nie w wierszu). Jeśli jednak ustawisz przełącznik `batch_first = True` to sieć rekurencyjna będzie się spodziewała wejścia jako `(rozmiar batcha, długość sekwencji, liczba cech)` czyli kolejne sekwencje będą w kolejnych wierszach tensora.

Dalej, warto zauważyć, że w przypadku np. warstwy liniowej do obliczenia wyniku wystarczyło podanie samych danych ( `linear(dane)`), jednak wejściem sieci rekurencyjnej są nie tylko dane ale i poprzednie stany ukryte $h_{t-1}$ ( `rnn(dane, h0)`).

In [ ]:
import torch
import torch.nn as nn

N_FEATURES = 2
N_LAYERS = 1
HIDDEN_SIZE = 3

data = torch.FloatTensor([[1, 1], [2, 2], [3, 3], [4, 4]]).view(1, 4, 2)
# Proste przykładowe dane: jedna 4 elementowa sekwencja gdzie każdy jej element jest opisany dwoma cechami
# Zwróć uwagę na wymiary tensora (rozmiar batcha, długość sekwencji, liczba cech elementów) 
#     - konieczne batch_first=True

rnn = nn.RNN(N_FEATURES, HIDDEN_SIZE, N_LAYERS, batch_first=True)

h0 = torch.randn(N_LAYERS, 1, HIDDEN_SIZE)
# Początkowy stan ukryty h0. Każda warstwa sieci rekurencyjnej zaczyna przetwarzanie od swojego h0
# W sytuacji gdy przetwarzamy kilka sekwencji na raz (sieć jest uruchamiana równolegle na kilku sekwencjach)
# również potrzebujemy dla każdej przetwarzanej sekwencji jej stan początkowy h0
# Porównaj powyższy opis z wymiarowością zmiennej h0

out, h_t = rnn(data, h0)
print("Wyjście warstwy: ", out)
print("Ostatni stan ukryty: ", h_t)


Zwróć uwagę na otrzymane wyjście: ponieważ na wejście sieci podaliśmy sekwencję elementów to dla każdego przetwarzanego elementu sekwencji sieć zwróciła uzyskaną reprezentację ukrytą (skoro mamy 3 neurony w warstwie to taka reprezentacja jest 3-wymiarowa). Dodatkowo zwrócony został ostatni stan ukryty, który moglibyśmy przetwarzać np. kolejnej sieci rekurencyjnej.

Sprawdź poprawność uzyskanego powyżej wyniku poprzez uruchomienie sieci rekurencyjnej element po elemencie (wejściem do sieci jest zawsze tylko jeden element sekwencji lub inaczej: sekwencje 1-elementowe). Wypisz na wyjście kolejne otrzymywane wyniki sieci i jej stany ukryte. Uzyskane wyniki powinny być takie same jak te z komórki wyżej.

In [ ]:
h1 = h0
print(h0)
for i in range(1, 5):
    x_i = torch.FloatTensor([[i, i]]).view(1, 1, 2)
    out, h1 = rnn(x_i, h1)
    print(out)

W prosty sposób można również rozszerzyć naszą warstwę rekurencyjną do sieci dwukierunkowej. W konstruktorze wystarczy ustawić `bidirectional = True`. Jak mówiliśmy na wykładzie taka sieć składa się w rzeczywistości z dwóch sieci - jednej przetwarzającej wyniki od lewej do prawej i drugiej przetwarzającej wyniki od prawej do lewej. W związku z tym należy dla obydwu tych sieci przygotować ich stany początkowe $h_0$, a wymiarowość wyjścia zwiększy się dwukrotnie.

In [ ]:
rnn = nn.RNN(N_FEATURES, HIDDEN_SIZE, N_LAYERS, batch_first=True, bidirectional=True)

h0 = torch.randn(N_LAYERS * 2, 1, HIDDEN_SIZE)
out, h_t = rnn(data, h0)
print("Wyjście warstwy: ", out)
print("Ostatni stan ukryty: ", h_t)


Analogicznie jak poprzednio uzyskaj wynik z poprzedniej komórki zakładając że na wejściu obiektu `rnn` możesz umieścić jedynie sekwencje jednoelementowe (możesz go jednak wywoływać dowolnie wiele razy).

In [ ]:
h1 = h0
for i in range(1, 5):
    x_i = torch.FloatTensor([[i, i]]).view(1, 1, 2)
    out, h1 = rnn(x_i, h1)
    print(out)

Na koniec warto też zauważyć, że samodzielna inicjalizacja stanu wejściowego do sieci rekurencyjnej `h0` nie jest obowiązkowa i na wejście warstwy rekurencyjnej można podać samą sekwencję wejściową. W takim wypadku element `h0` zostanie zainicjalizowany domyślnie wektorem zer.

In [ ]:
out, h_t = rnn(data)
print("Wyjście warstwy: ", out)
print("Ostatni stan ukryty: ", h_t)


**Ćwiczenia**
- Jakich zmian w kodzie musiałbyś dokonać by uzyskać 3-warstwową sieć rekurencyjną? Jak zmieniłaby się wtedy wymiarowośc `h0` oraz wyjścia sieci neuronowej `h1` i `out`?
- W przypadku neuronu rekurencyjnego RNN wejściem do modelu jest sekwencja oraz początkowy/poprzedni stan ukryty `h0`. Korzystając z gotowej implemetancji neuronu LSTM jakiego wejścia się spodziewasz?

Odpowiedzi nie musisz zapisywać.

## Zadanie 2
W ostatnim zadaniu domowym poznawałeś usprawnienia implementacji modeli uczących się w PyTorch poprzez wykorzystanie gotowych elementów z modułu `torch.nn`. Jednak ostatecznie uzyskany przez nas kod nadal przetwarzał pojedyncze instancje tj. nieobsługiwał mini-batchy. Można oczywiście ręcznie zaimplementować indeksowanie, które pozwoli nam na iterowanie po paczkach danych, jednak PyTorch pozwala oczywiście na automatyzację tego procesu.

Na początek wczytajmy dane uczące z których będziemy korzystać w tym zadaniu. (Może być konieczna instalacja biblioteki `torchtext`).

In [ ]:
from torchtext.datasets import CoNLL2000Chunking

train_iter = CoNLL2000Chunking(split='train')
seq_texts = []
seq_tags = []
for i in train_iter:
    seq_texts.append(i[0])
    seq_tags.append(i[2])
print(seq_texts[0])
print(seq_tags[0])

Są to dane z popularnego zbioru CoNLL2000 dotyczące zadania płytkiej analizy frazowej. W liście `seq_texts` umieszczono kolejne zdania, a na liście `seq_tags` umieszczono odpowiadające im zestawy tagów. Zwróć uwagę, że zastosowano tutaj tagowanie BIO np. frazą czasownikową `VP` jest "is widely expected to take".

Pierwszym krokiem przetwarzania jest zamiana słów na ich identyfikatory oraz obsłużenie tokenu `OOV`. W poprzednich zadaniach domowych robiliśmy to ręcznie jednak dzięki bibliotece `torchtext` ten proces równiez możemy zautomatyzować dzięki obiektowi `Vocab` i funkcji `build_vocab_from_iterator`.

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

vocab = build_vocab_from_iterator(seq_texts, specials=["<unk>"], min_freq=5)
len(vocab)

Funkcja ta na wejście przyjmuje iterator z tekstami, a także opcjonalnie próg `min_freq` pomijający słowa występujące z niewystarczającą częstotliowścią oraz `specials` pozwalający na umieszczenie w słowniku dodatkowych tokenów. Tokeny specjalne domyślnie są umieszczane na początku słownika.

Obiekt `Vocab` w prosty sposób zamienia sekwencję tokenów na sekwencję ich indeksów.

In [ ]:
vocab(["if", "could", "in", "<unk>"])

oraz na inne proste konwersje:

In [ ]:
print(vocab.lookup_token(107))
print(vocab.lookup_indices(["<unk>"]))

Domyślnie jednak słownik nie obsługuje słów spoza słownika. Dla przykładu "Confidence" występuje w rozważanym korpusie mniej niż 2 razy.

In [ ]:
#vocab(["Confidence"]) #TODO: Uncomment

Można jednak ustawić domyślny indeks słownika na token `UNK`, który automatycznie zamienia nieznane słowa na indeks tego tokenu.

In [ ]:
vocab.set_default_index(0)
vocab(["if", "could", "in", "<unk>", "Confidence"])

Następny krokiem jest implementacja własnej klasy zbioru danych, dziedziczącej po klasie `torch.utils.data.Dataset`, która zapewni kompatybilność reprezentacji naszych danych z procedurami m.in. automatycznie tworzącymi batche.

Zbiór danych powinien implementować co najmniej 3 funkcje: konstruktor, funkcję zwracającą liczbę elementów w zbiorze `__len__` oraz funckję pozwalającą na dostęp do wybranego elementu zbioru `__getitem__`.

Wykorzystując funkcję `build_vocab_from_iterator` uzupełnij implementację poniższej klasy, tak aby funkcja `__getitem__` zwracała elementy zbioru składające się z sekwencji indeksów słów oraz z sekwencji indeksów klas. Należy obsłużyć słowa spoza słownika zamieniająć na token `UNK` słowa występujące jednokrotnie w zbiorze danych. Dodatkowo słownik powinien uwzględniać token specjalny `<PAD>`, najlepiej umieszczony pod indeksem 0, który będzie potrzebny w dalszej części ćwiczenia.

In [ ]:
from torch.utils.data import Dataset, DataLoader


class CoNLLDataset(Dataset):
    def __init__(self, seq_texts, seq_tags):
        self.vocab_text = build_vocab_from_iterator(seq_texts, specials=["<pad>", "<unk>"], min_freq=2)
        self.vocab_text.set_default_index(1)

        self.vocab_tag = build_vocab_from_iterator(seq_texts, specials=["<pad>", "<unk>"], min_freq=2)
        self.vocab_tag.set_default_index(1)

        self.texts = [self.vocab_text(txt) for txt in seq_texts]
        self.labels = [self.vocab_tag(tag) for tag in seq_tags]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        return {'text': self.texts[item], 'label': self.labels[item]}

    def get_vocab_size(self):
        return len(self.vocab_text)

    def get_tagset_size(self):
        return len(self.vocab_tag)

In [ ]:
dataset = CoNLLDataset(seq_texts, seq_tags)
for i in dataset:
    print(i)
    break

Mając zaimplementowany zbiór danych jako obiekt typu `Dataset`, podzielenie go na paczki instancji nie jest trudne. Wystarczy wykorzystać obiekt `DataLoader` i wyspecyfikować w nim rozmiar paczki danych `batch_size`.

In [ ]:
dataloader = DataLoader(dataset, batch_size=4)
print("Ile utworzono paczek?", len(dataloader))

Niemniej jednak praca z tekstami nie jest niestety taka prosta spróbuj przeiterować pod kolejnych paczkach twojego zbioru:

In [ ]:
# for i in dataloader: TODO: UNCOMMENT
#     print(i)
#     break

Obiekt `DataLoader` próbuje automatycznie podzielić nasze dane na paczki, jednak nie jest to takie proste. Nasze dane składają się z sekwencji o różnych długościach, ciężko jest więc utworzyć z nich eleganckie tensory które mają stałe wymiarowości. Naturalnym rozwiązaniem problemu jest wyznaczenie długości najdłuższej sekwencji i uzupełnienie wszystkich pozostałych sekwencji do tej długości specjlanymi tokenami `<PAD>`. Taka operacja może niestety znacznie zwiększyć wymagania pamięciowe potrzebne do reprezentacji zbioru. Pojawienie się jednej bardzo długiej sekwencji w zbiorze znacznie zwiększa czas przetwarzania dla wszystkich jego elementów. 

Zauważ, że wymaganiem technicznym nie jest posiadanie całego zbioru danych w postaci jednego dużego tensora, ale stworzenie takiego tensora dla wszystkich sekwnencji w ramach jednej paczki danych. Zwykle długość najdłużej sekwencji w paczce jest dużo mniejsza niż długość najdłuszej sekwencji w całym zbiorze, co pozwoliłoby nam na nie tylko znaczne lepsze wykorzystanie pamięci operacyjnej, ale także na ograniczenie czasu przetwarzania. Z tego powodu w tym zadaniu będziemy dynamicznie tworzyć reprezentację paczki danych -- każda paczka będzie zawierała tyle samo sekwencji, jednak będą one uzupełniane do różnych długości, pod tym względem paczki danych *nie* będą równe.  

Do uzupełnienia sekwencji do równych długości przydatna będzie funkcja `pad_sequence`, która również posiada parametr `batch_first` przygotowując dane o odpowiednich wymiarach.

In [ ]:
from torch.nn.utils.rnn import pad_sequence

data = [torch.tensor([1, 2, 3]), torch.tensor([1, 2])]
pad_data = pad_sequence(data, padding_value=0)
print(pad_data)

pad_data = pad_sequence(data, padding_value=0, batch_first=True)
print(pad_data)

Warto zwrócić uwagę, że zrównoleglanie przetwarzania sieci rekurencyjnej nie jest proste, gdyż wyniki przetwarzania zależą od poprzednich wyników. Nie można więc, tak jak w przypadku sieci splotowej, równocześnie obliczyć wyników filtra na wszystkich słowach w tekście, znakomicie zrównoleglając przetwarzanie. Należy najpierw zastosować "filtr" na pierwszym słowie, poczekać na wynik, potem na kolejnym itd. W związku z tym najlepszą możliwością zrównoleglenia przetwarzania sieci rekurencyjnych jest obsługa wielu przykładów uczących na raz. Sieć rekurencyjna, inicjalizowana potencjalnie różnymi `h0` dla różnych przykładów uczących, na raz przetwarza pierwsze elementy każdej sekwencji. Następnie przetwarza wszystkie drugie elementy, wszystkie trzecie elementy itd. Przetwarzanie odbywać się będzie po kolei w ramach kolejnych wierszy domyślnej repezentacji `batch_first=False` tj. równocześnie przetwarzane będą wszystkie sekwencje w batchu.

Implementacja dynamicznego tworzenia paczek danych (wraz z uzupełnianiem `<PAD>`) jest możliwa dzięki przekazaniu do konstruktora `DataLoader` funkcji `collate_fn`. Funkcja ta na wejście otrzymuje kolekcję danych (w takiej postaci w jakiej zostały one zwrócone z `DataSet` a na wyjście powinna zwrócić tensory reprezentujące paczkę danych. Nasza funkcja `collate_fn` powinna zwrócić 3 tensory. Pierwszy tensor powinien zawierać przykłady uczące (o równej długości), drugi odpowiednio uzupełnione sekwencje tagów oraz trzeci tensor jednowymiarowy (wektor) zawierający długości kolejnych sekwencji w batchu.

*UWAGA*: Konwencją umożliwiającą potem efektywniejsze przetwarzanie jest to, że sekwencje w batchu są posortowane ich długościami. Pierwsza sekwencja powinna być najdłuższa, a ostatnia najkrótsza.

In [ ]:
def collate_fn(batch):
    batch.sort(key=lambda x: len(x['text']), reverse=True)

    txt_tensor = []
    lbl_tensor = []
    info_tensor = []

    for b in batch:
        txt_tensor.append(torch.tensor(b['text']))
        lbl_tensor.append(torch.tensor(b['label']))
        info_tensor.append(len(b['text']))

    # txt_tensor = [torch.tensor(x['text']) for x in batch]
    # lbl_tensor = [torch.tensor(x['label']) for x in batch]
    # info_tensor = [len(x) for x in txt_tensor]

    txt_tensor = pad_sequence(txt_tensor, padding_value=0, batch_first=True)
    lbl_tensor = pad_sequence(lbl_tensor, padding_value=0, batch_first=True)

    return txt_tensor, lbl_tensor, info_tensor

In [ ]:
dataloader = DataLoader(dataset, batch_size=4, collate_fn=collate_fn)
for i in dataloader:
    print(i)
    break

Uzupełnienie sekwencji tokenami `<PAD>` do pełnej długości wydaje sie być dobrym pomysłem i pozwala na operowanie na tensorach które łatwo przesłać na kartę graficzną czy wykonać równoległe obliczenia na wszystkich jego elementach. Niemniej jednak fakt, że nasze sekwencje nie są równiej długości jest nadal niezwykle istotny przy wielu różnych obliczeniach. W szczególności jest on istotny przy przetwarzaniu sekwencji przez sieć rekurencyjną. W przypadku sieci rekurencyjnej przetwarzającej sekwencję uzupełnioną do końca zerami, sieć nie zwróci reprezentacji dla każdego słowa ale dla wszystkich elementów sekwencji czyli także dla końcowych elementów-zer. Ewidentnie sieć wykonała wiele niepotrzebnych operacji obliczeniowych tracąc czas (a przecież sieci rekurencyjne do najszybszych nie należą) jednakże prawidłowy wynik nadal jest do odzyskania. Możemy przecież post-factum wziąć pod uwagę tylko (długość sekwencji)-pierwszych elementów wyniku, uzyskując taki sam wynik jak przy przetwarzaniu sekwencji bez uzupełnienia.

Tak się jednak nie dzieje, gdy rozważamy dwukierunkową sieć rekurencyjną. Sieć iterująca w tył rozpoczęła przetwarzanie od początkowego stanu ukrytego, a następnie akutalizowała go za kolejne wejścia uważając sekwencję tokenów `<PAD>`! Wynik obliczeń tej sieci jest więc różny niż dla sekwencji nieuzupełnionej i co więcej nie jest on do odzyskania. Uzupełniając sekwnecję np. zerami nie tylko wykonujemy nadmiarowe obliczenia, ale także zaburzamy wyniki.

Na szczęście sieci rekurencyjne zaimplementowane w PyTorch obsługją także specjalny format danych tzw. `PackedSequence`, który przechowuje informację o długościach sekwencji zapewniając oszczędność obliczeń i takie same wyniki jak dla sekwencji bez uzupełniania. Przed wykonaniem obliczeń siecią rekurencyjną możemy dane zapakować do tego formatu, a następnie przetworzony wynik możemy z powrotem odpakować do postaci uzupełnionego do pełnej długości tensora. Pomocne są w tym dwie funkcje: `pack_padded_sequence` pakująca dane do tego formatu oraz funkcja odpakowująca `pad_packed_sequence`.

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

data = torch.tensor([[1, 2, 3, 4, 5], [-1, -2, -3, 0, 0], [10, 20, 0, 0, 0]])
lengths = torch.tensor([5, 3, 2])

packed_data = pack_padded_sequence(data, lengths, batch_first=True)
print(packed_data)

Zapakowaliśmy 3 sekwencje, które uzupełnione były do długości 5. Ponieważ kolejne sekwencje są umieszczone w kolejnych wierszach tensora należy wyspecyfikować argument `batch_first=True`. Format `PackedSequence` przechowuje tensor `data` oraz tensor `batch_sizes`. Tak jak opisywaliśmy, sieć rekurencyjna zrównolegla swoje działania poprzez jednoczene przetwarzanie wszystkich sekwencji na raz, iterując po ich kolejnych elementach. Z tego powodu `batch_sizes` opisują wielkość przetwarzanej paczki danych (tj. liczba przetwarzanych sekwencji) w każdej iteracji. Na początku przetwarzamy 3 sekwencje, potem znowu 3, a następnie tylko dwie, gdyż ostatnia sekwencja miała tylko 2 elementy. Prześledź, że właśnie w taki sposób zostały ułożone dane w `data`.

Porównajmy działanie neuronu rekurencyjnego dla danych uzupełnionych tokenami `<PAD>` oraz dla danych spakowanych do `PackedSequence`.

In [ ]:
N_FEATURES = 1
N_LAYERS = 1
HIDDEN_SIZE = 1

data = torch.FloatTensor([[1, 2, 3, 4, 5], [-1, -2, -3, 0, 0], [10, 20, 0, 0, 0]]).view(3, 5, 1)
lengths = torch.tensor([5, 3, 2])

rnn = nn.RNN(N_FEATURES, HIDDEN_SIZE, N_LAYERS, batch_first=True)
out, _ = rnn(data)
print("Wyjście sieci dla danych o stałej długości: ", out)

packed_data = pack_padded_sequence(data, lengths, batch_first=True)
print(data.shape, packed_data[0].shape)
out_packed, _ = rnn(packed_data)
print("Wyjście sieci dla danych spakowanych: ", out_packed)
out, out_len = pad_packed_sequence(out_packed, batch_first=True)
print("Wyjście sieci po rozpakowaniu: ", out)


Zwróć uwagę jak ułożone są spakowane wyniki sieci.

Stworzyliśmy zbiór danych, mamy także zaimplementowany mechanizm dynamicznych paczek danych, a także wiemy jak efektywnie wykorzystywać je do obliczeń siecami rekurencyjnymi. Połóżmy więc wisienkę na torcie i stwórzmy model do tagowania tych danych. Model powinien składać się z warstwy zanurzeń słów, która jest wejściem do jednowarstwowego, dwukierunkowego LSTM. Ostatecznie predykcja jest wykonywana przez warstwę liniową (softmax).

In [ ]:
WORD_EMBEDDING = 20


class TaggerNet(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_tags):
        """ Argumentami konstruktora jest rozmiar słownika, 
            liczba neuronów w warstwie rekurencyjnej 
            oraz liczba klas/tagów
        """
        super(TaggerNet, self).__init__()
        self.embedding = nn.Embedding(vocab_size, WORD_EMBEDDING)

        self.lstm = nn.LSTM(WORD_EMBEDDING, hidden_size, n_tags, batch_first=True, bidirectional=True)
        self.soft = nn.Softmax()

    def forward(self, sentence, seq_lengths):
        """ Wejściem jest paczka zdań (słowa reprezentowane przez indeksy)
            oraz tensor ich długości 
        """
        emb = self.embedding(sentence)
        print("EMB")

        packed_data = pack_padded_sequence(emb, seq_lengths, batch_first=True)
        out_packed, h = self.lstm(packed_data)
        print("LSTM")
        out, out_len = pad_packed_sequence(out_packed, batch_first=True)

        out = self.soft(out)
        return out


from torch.optim import Adam

model = TaggerNet(dataset.get_vocab_size(), 10, dataset.get_tagset_size())
dataloader = DataLoader(dataset, batch_size=20, collate_fn=collate_fn)

loss_function = nn.CrossEntropyLoss(ignore_index=0)
optimizer = Adam(model.parameters())

for epoch in range(5):
    loss_sum = torch.tensor(0.)
    for sentences, tags, lengths in dataloader:
        pred = model(sentences, lengths)
        # print(pred.shape, tags.shape)
        # pred = pred.mean(dim=0)
        # pred = torch.reshape(pred, (pred.shape[0], pred.shape[1]))
        pred = pred.swapaxes(1, 2)
        # print(pred.shape, tags.shape)

        loss = loss_function(pred, tags)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), 1.5)

        optimizer.step()
        optimizer.zero_grad()

        loss_sum += loss
    print(loss_sum / len(dataloader))

Zaimplemetuj finalną pętlę uczącą model. 
- Powinieneś wykorzystać błąd entropii krzyżowej jako funkcję straty policzoną dla każdego przewidzianego taga. 
- Zwróć uwagę, że wyniki sieci są w postaci rozpakowanej tj. uzupełnionej zerami do pełnej długości. Błąd nie powinien być liczony dla tych predykcji! Możesz to osiągnąć np. sprytnie wykorzystując parametr `ignore_index` klasy `nn.CrossEntropyLoss`. 
- Jako algorytm optymalizacyjny wykorzystaj `torch.optim.Adam`.
- Jedną z technik stabilizujących trening sieci jest przycinanie gradientu. W PyTorch możesz to uzyskać wywołując pomiędzy obliczeniem gradientu a wywołaniem kroku optymalizatora funkcji `nn.utils.clip_grad_norm_(model.parameters(), TRESHOLD)`. Zwróć uwagę, że nazwa funkcji zakończona jest `_` to znaczy, że operacja jest wykonywana in-place i wyniku funkcji nie trzeba podstawiać do żadnej zmiennej.
- Celem ćwiczenia nie jest wybór hiperparametrów, ani uzyskiwanie wysokich trafności.

In [ ]:
# from torch.optim import Adam
#
# model = TaggerNet(dataset.get_vocab_size(), 10, dataset.get_tagset_size())
# dataloader = DataLoader(dataset, batch_size=20, collate_fn=collate_fn)
#
# loss_function = nn.CrossEntropyLoss(ignore_index=0)
# optimizer = Adam(model.parameters())
#
# for epoch in range(5):
#     loss_sum = torch.tensor(0.)
#     for sentences, tags, lengths in dataloader:
#         pred = model(sentences, lengths)
#
#         loss = loss_function(pred, tags)
#         loss.backward()
#
#         # nn.utils.clip_grad_norm_(model.parameters())
#
#         optimizer.step()
#         optimizer.zero_grad()
#
#         loss_sum += loss
#     print(loss_sum / len(dataloader))
#


**Ćwiczenia**
- W tym zadaniu zaimplementowaliśmy dynamiczne tworzenie paczek danych, w ramach których uzupełnialiśmy sekwencje do takiej samej długości. Czasami stosowaną techniką jest posortowanie zbioru danych, tak aby sekwencje były ułożone od najdłuższej do najkrótszej, a następnie dynamicznie tworzy się kolejne paczeki danych iterując po tak ułożonym zbiorze. Jakie są wady i zalety takiego rozwiązania?
- Student podczas treningu sieci neuronowej zauważył, że wyniki funkcji celu w czasie optymalizacji modelu nie są stabilne. Spodziewa się, że zwiększenie rozmiaru paczki danych przyczyni się do bardziej stabilnego treningu sieci i poprawi wyniki. Jednakże, dalsze zwiększenie paczki danych nie jest możliwe gdyż nie mieści się ona w pamięci na karcie GPU. Sama implementacja sieci i reprezentacji zbioru danych jest wg. studenta optymalna, inna karta GPU o większej pamięci nie jest dostępna. Jak rozwiązac ten problem? Podaj zarys implementacji. 


Odpowiedź na ostatnią kropkę umieść poniżej.


TWOJA ODWPOWIEDŹ TUTAJ